# Imports

In [ ]:
import numpy as np
import xarray as xr
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Polygon
from matplotlib import colors as mat_colors
import mpl_toolkits.axisartist as axisartist
from mpl_toolkits.axes_grid1 import Size, Divider

# Define Functions

## Performance measurements

In [ ]:
def BIAS(a1, a2):
    return (a1 - a2).mean().item()


def RMSE(a1, a2):
    return np.sqrt(((a1 - a2)**2).mean()).item()


def DIFF(a1, a2):
    return np.max(np.abs(a1 - a2)).item()

## help function for heatmap axis

In [ ]:
def setup_axes(fig, rect):
    ax = axisartist.Subplot(fig, rect)
    fig.add_subplot(ax)

    return ax

## heatmap

In [ ]:
def heatmap(datasets, # first_dataset, second_dataset,
            opti_var,
            annotation=None,
            annotation_x_position=0,
            fig=None, ax=None,
            cmap='vlag',
            cmap_levels=None,
            grid_color='grey',
            grid_linewidth=1.5,
            presentation=False,
            labels_pad=-360,
            xlim=None,
            nr_of_iterations=None):
    if not ax:
        ax = plt.gca()
    
    if not fig:
        fig = plt.gcf()

    if all(dataset is None for dataset in datasets):
        raise ValueError('All datasets are None!')

    # define variables for plotting
    guess_opti_var = []
    first_guess_diff = []
    true_opti_var = []
    BIAS_opti_var = []
    RMSE_opti_var = []
    DIFF_opti_var = []
    fct_opti_var = []
    times = []
    maxiters = []
    BIAS_sfc = []
    RMSE_sfc = []
    DIFF_sfc = []
    BIAS_w = []
    RMSE_w = []
    DIFF_w = []
    BIAS_fg = []
    RMSE_fg = []
    DIFF_fg = []
    BIAS_sfc_fg = []
    RMSE_sfc_fg = []
    DIFF_sfc_fg = []
    array_length = 0
    check_first_guess = None
    check_true_opti_var = None

    # create data and label variables
    for dataset in datasets:
        # check if the current dataset contains data or if the data was not available
        if dataset is None:
            guess_opti_var.append(None)
            first_guess_diff.append(None)
            true_opti_var.append(None)
            BIAS_opti_var.append(None)
            RMSE_opti_var.append(None)
            DIFF_opti_var.append(None)
            fct_opti_var.append(None)
            times.append(None)
            maxiters.append(None)
            BIAS_sfc.append(None)
            RMSE_sfc.append(None)
            DIFF_sfc.append(None)
            BIAS_w.append(None)
            RMSE_w.append(None)
            DIFF_w.append(None)
        elif type(dataset) != xr.core.dataset.Dataset:  # if no minimisation possible
            guess_opti_var.append('no_minimisation')
            first_guess_diff.append(None)
            true_opti_var.append(None)
            BIAS_opti_var.append(None)
            RMSE_opti_var.append(None)
            DIFF_opti_var.append(None)
            fct_opti_var.append(None)
            times.append(None)
            maxiters.append(None)
            BIAS_sfc.append(None)
            RMSE_sfc.append(None)
            DIFF_sfc.append(None)
            BIAS_w.append(None)
            RMSE_w.append(None)
            DIFF_w.append(None)
        else:
            # find index corresponding to max time
            max_index = len(dataset['computing_time'].values) - 1

            if nr_of_iterations is not None:
                max_index = nr_of_iterations - 1
            elif xlim is not None:
                # only consider as many points until max of xlim is reached
                if dataset['computing_time'].values[-1] > xlim[1]:
                    max_index = np.argmax(dataset['computing_time'].values > xlim[1])

            if opti_var == 'bed_h':
                guess_opti_var.append((dataset.guessed_bed_h[max_index] - dataset.true_bed_h).values)
                first_guess_diff.append((dataset.first_guessed_bed_h - dataset.true_bed_h).values)
                true_opti_var.append(dataset.true_bed_h.values)
                BIAS_opti_var.append(BIAS(dataset.guessed_bed_h[max_index], dataset.true_bed_h))
                RMSE_opti_var.append(RMSE(dataset.guessed_bed_h[max_index], dataset.true_bed_h))
                DIFF_opti_var.append(DIFF(dataset.guessed_bed_h[max_index], dataset.true_bed_h))
                if check_first_guess is None:
                    BIAS_fg = BIAS(dataset.first_guessed_bed_h, dataset.true_bed_h)
                    RMSE_fg = RMSE(dataset.first_guessed_bed_h, dataset.true_bed_h)
                    DIFF_fg = DIFF(dataset.first_guessed_bed_h, dataset.true_bed_h)
            elif opti_var == 'bed_shape':
                guess_opti_var.append((dataset.guessed_bed_shape[-1] - dataset.true_bed_shape).values)
                first_guess_diff.append((dataset.first_guessed_bed_shape - dataset.true_bed_shape).values)
                true_opti_var.append(dataset.true_bed_shape.values)
                BIAS_opti_var.append(BIAS(dataset.guessed_bed_shape[max_index], dataset.true_bed_shape))
                RMSE_opti_var.append(RMSE(dataset.guessed_bed_shape[max_index], dataset.true_bed_shape))
                DIFF_opti_var.append(DIFF(dataset.guessed_bed_shape[max_index], dataset.true_bed_shape))
                if check_first_guess is None:
                    BIAS_fg = BIAS(dataset.first_guessed_bed_shape, dataset.true_bed_shape)
                    RMSE_fg = RMSE(dataset.first_guessed_bed_shape, dataset.true_bed_shape)
                    DIFF_fg = DIFF(dataset.first_guessed_bed_shape, dataset.true_bed_shape)
            elif opti_var == 'w0':
                guess_opti_var.append((dataset.guessed_w0[-1] - dataset.true_w0).values)
                first_guess_diff.append((dataset.first_guessed_w0 - dataset.true_w0).values)
                true_opti_var.append(dataset.true_w0.values)
                BIAS_opti_var.append(BIAS(dataset.guessed_w0[max_index], dataset.true_w0))
                RMSE_opti_var.append(RMSE(dataset.guessed_w0[max_index], dataset.true_w0))
                DIFF_opti_var.append(DIFF(dataset.guessed_w0[max_index], dataset.true_w0))
                if check_first_guess is None:
                    BIAS_fg = BIAS(dataset.first_guessed_w0, dataset.true_w0)
                    RMSE_fg = RMSE(dataset.first_guessed_w0, dataset.true_w0)
                    DIFF_fg = DIFF(dataset.first_guessed_w0, dataset.true_w0)
            else:
                raise ValueError('Unknown opti var!')
                
            fct_opti_var.append(dataset.function_calls[max_index].values)
            times.append(dataset.computing_time[max_index].values)
            maxiters.append(dataset.attrs['maxiter_reached'])
            BIAS_sfc.append(BIAS(dataset.surface_h[max_index], dataset.true_surface_h))
            RMSE_sfc.append(RMSE(dataset.surface_h[max_index], dataset.true_surface_h))
            DIFF_sfc.append(DIFF(dataset.surface_h[max_index], dataset.true_surface_h))
            BIAS_w.append(BIAS(dataset.widths[max_index], dataset.true_widths))
            RMSE_w.append(RMSE(dataset.widths[max_index], dataset.true_widths))
            DIFF_w.append(DIFF(dataset.widths[max_index], dataset.true_widths))

            # determine array length for empty lines
            if array_length == 0:
                array_length = dataset.points_with_ice[-1].values + 1
            # check that the arrays have the same number of points with ice
            elif array_length != dataset.points_with_ice[-1].values + 1:
                raise ValueError('Not the same lentgth of points with ice!!!')

            # check if all experiments start with the same true values and first guess
            # in the first round save values
            if check_first_guess is None:
                check_first_guess = first_guess_diff[-1]
                check_true_opti_var = true_opti_var[-1]
                
                # not implemented yet
                BIAS_sfc_fg = BIAS(dataset.first_guess_surface_h, dataset.true_surface_h)
                RMSE_sfc_fg = RMSE(dataset.first_guess_surface_h, dataset.true_surface_h)
                DIFF_sfc_fg = DIFF(dataset.first_guess_surface_h, dataset.true_surface_h)
                BIAS_w_fg = BIAS(dataset.first_guess_widths, dataset.true_widths)
                RMSE_w_fg = RMSE(dataset.first_guess_widths, dataset.true_widths)
                DIFF_w_fg = DIFF(dataset.first_guess_widths, dataset.true_widths)

            # after first round compare all values to first ones to make sure comparing the same start conditions
            else:
                if np.sum(check_true_opti_var - true_opti_var[-1]) != 0:
                    raise ValueError('Not the same true control variable!!!')
                if np.sum(check_first_guess - first_guess_diff[-1]) != 0:
                    raise ValueError('Not the same first guess!!!')

    # create variables for ploting (data and y label)
    data = []
    y_labels = []

    # first add heading
    data.append(np.empty((array_length)) * np.nan)
    if not presentation:
        if opti_var == 'bed_h':
            y_labels.append('    RMSE_b  DIFF_b  RMSE_s  DIFF_s  fct  time')
        elif opti_var in ['bed_shape', 'w0']:
            y_labels.append('    RMSE_b  DIFF_b  RMSE_w  DIFF_w  fct  time')
        else:
            raise ValueError('Unknown opti_var !')
        y_label_variable_format = '{:6.2f}, {: 6.2f}, {:6.2f}, {:6.2f}'
    else:
        if opti_var == 'bed_h':
            y_labels.append(r'   DIFF_b, fct, $T_{cpu}$')
        elif opti_var in ['bed_shape', 'w0']:
            y_labels.append('     DIFF  DIFF_w  fct  time')
        else:
            raise ValueError('Unknown opti_var !')
        y_label_variable_format = '{: 6.2f}' #', {:6.2f}'

    if not presentation:
        # add first guess
        data.append(check_first_guess)
        if opti_var == 'bed_h':
            y_labels.append(('fg:' + y_label_variable_format).format(RMSE_fg, DIFF_fg,
                                                                     RMSE_sfc_fg, DIFF_sfc_fg))
        elif opti_var in ['bed_shape', 'w0']:
            y_labels.append(('fg:' + y_label_variable_format).format(RMSE_fg, DIFF_fg,
                                                                     RMSE_w_fg, DIFF_w_fg))
        else:
            raise ValueError('Unknown opti_var !')
    else:
        # add first guess
        data.append(check_first_guess)
        if opti_var == 'bed_h':
            y_labels.append(('fg:' + y_label_variable_format).format(DIFF_fg))
        elif opti_var in ['bed_shape', 'w0']:
            y_labels.append(('fg:' + y_label_variable_format).format(DIFF_fg, DIFF_w_fg))
        else:
            raise ValueError('Unknown opti_var !')
    
    # add two format placeholders for fct_calls and time
    y_label_variable_format += ', {:2d}, {:3.0f}s'

    # add all other data with empty line for None datasets
    for i, guess in enumerate(guess_opti_var):
        if guess is None:
            data.append(np.empty((array_length)) * np.nan)
            if i < 9:
                y_labels.append((' ' + str(i+1) + ':    NO DATAFILE FOUND'))
            else:
                y_labels.append((str(i+1) + ':    NO DATAFILE FOUND'))
        elif type(guess) is str:
            data.append(np.empty((array_length)) * np.nan)
            if i < 9:
                y_labels.append((' ' + str(i+1) + ':    NO Minimisation Possible'))
            else:
                y_labels.append((str(i+1) + ':    NO Minimisation Possible'))
        else:
            data.append(guess)
            if i < 9:
                y_label_text = (' ' + str(i+1) + ':' + y_label_variable_format)
            else:
                y_label_text = (str(i+1) + ':' + y_label_variable_format)
            
            if maxiters[i] == 'yes':
                y_label_text += '+'

            if opti_var == 'bed_h':
                if not presentation:
                    y_labels.append(y_label_text.format(RMSE_opti_var[i],
                                                        DIFF_opti_var[i],
                                                        RMSE_sfc[i],
                                                        DIFF_sfc[i],
                                                        fct_opti_var[i],
                                                        times[i]))
                else:
                    y_labels.append(y_label_text.format(DIFF_opti_var[i],
                                                        fct_opti_var[i],
                                                        times[i]))
            elif opti_var in ['bed_shape', 'w0']:
                if not presentation:
                    y_labels.append(y_label_text.format(RMSE_opti_var[i],
                                                        DIFF_opti_var[i],
                                                        RMSE_w[i],
                                                        DIFF_w[i],
                                                        fct_opti_var[i],
                                                        times[i]))
                else:
                    y_labels.append(y_label_text.format(DIFF_opti_var[i],
                                                        DIFF_w[i],
                                                        fct_opti_var[i],
                                                        times[i]))
            else:
                raise ValueError('Unknown opti_var !')

    # make data an numpy array
    data = np.array(data)

    #choose colormap
    if not cmap_levels:
        color_nr = 100
        if opti_var == 'bed_h':
            cmap_limit = np.max(np.abs(check_first_guess))
            #cmap_limit = np.max(np.array([np.abs(np.floor(np.nanmin(np.array(data)))),
            #                              np.abs(np.ceil(np.nanmax(np.array(data))))]))
        elif opti_var in ['bed_shape', 'w0']:
            cmap_limit = np.max(np.abs(check_first_guess))
            #cmap_limit = np.max(np.array([np.abs(np.floor(np.nanmin(np.array(data)) * 10)),
            #                              np.abs(np.ceil(np.nanmax(np.array(data)) * 10))])) / 10
        else:
            raise ValueError('Unknown opti var!!')
        #if (np.min(data) < 0) & (np.max(data) > 0):
        cmap_levels = np.linspace(-cmap_limit, cmap_limit, color_nr, endpoint=True)
        #elif (np.min(data) < 0) & (np.max(data) =< 0):
        #    cmap_levels = np.linspace(-cmap_limit, 0, color_nr, endpoint=True)
        #elif (np.min(data) >= 0) & (np.max(data) > 0)
    else:
        color_nr = len(cmap_levels) - 1
    
    rel_color_steps = np.arange(color_nr)/color_nr
    if cmap == 'rainbow':
        colors = cm.rainbow(rel_color_steps)
    elif cmap == 'vlag':
        colors = sns.color_palette('vlag', color_nr)
    elif cmap == 'icefire':
        colors = sns.color_palette('icefire', color_nr)
    elif cmap == 'Spectral':
        colors = sns.color_palette('Spectral_r', color_nr)
    
    cmap = LinearSegmentedColormap.from_list('custom', colors, N=color_nr)
    cmap.set_bad(color='white')
    norm = mat_colors.BoundaryNorm(cmap_levels, cmap.N)

    # plot heatmap
    im = plt.imshow(data, aspect='auto', interpolation=None, cmap=cmap, norm=norm, alpha=1.)
    
    # Turn spines and ticks off and create white frame.
    for key, spine in ax.axis.items():
        spine.major_ticks.set_visible(False)
        spine.minor_ticks.set_visible(False)
        spine.line.set_visible(False)
        # spine.line.set_color(grid_color)
        # spine.line.set_linewidth(0) #grid_linewidth)
    
    # set y ticks
    ax.set_yticks(np.arange(data.shape[0]))
                
    ax.set_yticklabels(y_labels)
    #for tick in ax.get_yticklabels():
    #    tick.set_fontname("Arial")

    # align yticklabels left
    ax.axis["left"].major_ticklabels.set_ha("left")
    
    # set pad to put labels over heatmap
    ax.axis["left"].major_ticklabels.set_pad(labels_pad)

    # set y minor grid
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", axis='y', color=grid_color, linestyle='-', linewidth=grid_linewidth)
    
    # set x ticklabels off
    ax.set_xticklabels([])
    
    # create colorbar
    cax = ax.inset_axes([1.01, 0.1, 0.03, 0.8]) 
    #cax = fig.add_axes([0.5, 0, 0.01, 1])
    cbar = fig.colorbar(im, cax=cax, boundaries=cmap_levels, spacing='proportional',)
    cbar.set_ticks([np.min(cmap_levels),0,np.max(cmap_levels)])
    if opti_var == 'bed_h':
        cbar.set_ticklabels(['{:d}'.format(int(-cmap_limit)), '0' ,'{:d}'.format(int(cmap_limit))])
    elif opti_var == 'bed_shape':
        cbar.set_ticklabels(['{:.1f}'.format(-cmap_limit), '0' ,'{:.1f}'.format(cmap_limit)])
    elif opti_var == 'w0':
        cbar.set_ticklabels(['{:d}'.format(int(-cmap_limit)), '0' ,'{:d}'.format(int(cmap_limit))])
    else:
        raise ValueError('Unknown opti var!!')
    #cbar.ax.set_ylabel(cbarlabel,)
    
    # set title
    #ax.set_title(title)
    
    if annotation is not None:
        # include text
        ax.text(annotation_x_position, 1.05, 
                 annotation,
                 horizontalalignment='left',
                 verticalalignment='center',
                 transform=ax.transAxes)
    
    return im

## convert iterations to datasets

In [ ]:
def convert_iterations_to_dataset(dataset, timesteps=1):
    #total_iterations = dataset.coords['nr_of_iteration'].values
    #relative_iteration = int(total_iterations[-1] / (timesteps - 1))
    #plot_iterations = total_iterations[::relative_iteration]
    #if len(plot_iterations) == timesteps - 1:
    #    plot_iterations= np.append(plot_iterations, total_iterations[-1])
    #elif len(plot_iterations) == timesteps:
    #    plot_iterations[-1] = total_iterations[-1]
    #else:
    #    raise ValueError('Something went wrong with calculation of timesteps of iterations!')
    plot_iterations = dataset.coords['nr_of_iteration'].values

    timeseries_datasets = []
    for iteration in plot_iterations - 1:
        tmp_dataset = xr.Dataset(
                    data_vars={
                        'true_bed_h':
                            (['points_with_ice'],
                             dataset.true_bed_h),
                        'first_guessed_bed_h':
                            (['points_with_ice'],
                             dataset.first_guessed_bed_h),
                        'guessed_bed_h':
                            (['nr_of_iteration', 'points_with_ice'],
                             np.array([dataset.guessed_bed_h[iteration].values])),
                        'function_calls':
                            (['nr_of_iteration'],
                             np.array([dataset.function_calls[iteration].values])),
                        'true_surface_h':
                            (['total_distance'],
                             dataset.true_surface_h),
                        'first_guess_surface_h':
                            (['total_distance'],
                             dataset.first_guess_surface_h),
                        'surface_h':
                            (['nr_of_iteration', 'total_distance'],
                             np.array([dataset.surface_h[iteration].values])),
                        'true_widths':
                            (['total_distance'],
                             dataset.true_widths),
                        'first_guess_widths':
                            (['total_distance'],
                             dataset.first_guess_widths),
                        'widths':
                            (['nr_of_iteration', 'total_distance'],
                             np.array([dataset.widths[iteration].values])),
                        'computing_time':
                            (['nr_of_iteration'],
                             np.array([dataset.computing_time[iteration].values]))
                    },
                    coords={
                        'total_distance': dataset.coords['total_distance'],
                        'points_with_ice': dataset.coords['points_with_ice'],
                        'nr_of_iteration': ([1])
                    },
                    attrs={
                        #'computing_time': -1,
                        'maxiter_reached': False
                    }
        )
        #if iteration == plot_iterations[-1] - 1:
        #    tmp_dataset.attrs['computing_time'] = dataset.attrs['computing_time']
        timeseries_datasets.append(tmp_dataset)
    return timeseries_datasets

## performance plot

In [ ]:
def performance_plot(ax,
                     dataset,
                     nr_of_iterations,
                     annotation=None,
                     annotation_x_position=-0.2,
                     lw=2,
                     fontsize=25,
                     ms=10):
    x_use = np.arange(0, nr_of_iterations + 1)
    
    colors = [color_1, color_2, color_3, color_4]
    labels = ['50 a', '100 a', '150 a', '200 a']
    
    for i, dataset in enumerate(datasets):
        true_spinup_ELA = dataset.true_spinup_ELA.data
        
        # calculate first guess ELA diff
        ELA_diff_fg = dataset.first_guess_spinup_ELA.data #- true_spinup_ELA

        # calculate ELA diff and put together
        ELA_diff = np.append(ELA_diff_fg,
                             dataset.spinup_ELA_guessed.data[:nr_of_iterations]) # - true_spinup_ELA)

        # plot curves
        ax.plot(x_use, ELA_diff,
                '.-',
                c=colors[i],
                lw=lw,
                ms=ms,
                label=labels[i],
                zorder=4)
    
    # add zero line
    #plt.axhline(0,
    #           ls='--',
    #           c=axis_color,
    #           lw=lw,
    #           zorder=1)
    #ax.plot([x_use[0], x_use[-1]],
    #        [0, 0],
    #        '--',
    #        lw=lw,
    #        c=axis_color,
    #        zorder=1)
    
    ax.set_xlabel('Iteration', fontsize=fontsize, c=axis_color)
    ax.set_ylabel('height (m)', fontsize=fontsize, c=axis_color)
    
    #[t.set_color(axis_color) for t in ax.xaxis.get_ticklines()]
    #[t.set_color(axis_color) for t in ax.xaxis.get_ticklabels()]
    ax.tick_params(axis='both', colors=axis_color, width=lw)
    ax.spines['bottom'].set_color(axis_color)
    ax.spines['bottom'].set_linewidth(lw)
    ax.spines['left'].set_color(axis_color)
    ax.spines['left'].set_linewidth(lw)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend(fontsize=fontsize)
    
    if annotation is not None:
        ax.text(annotation_x_position, 1.1,
                annotation,
                horizontalalignment='left',
                verticalalignment='center',
                transform = ax.transAxes)

## function for profil

In [ ]:
def profil_subplot(
            ax,
            true_bed_h,
            true_sfc_h,
            second_bed_h,
            second_sfc_h,
            title,
            fontsize,
            tick_labels=None,
            tick_labels_indexes=None,
            tick_labels_position='above',
            len_tick_label_line = 110,
            tick_label_text_y_distance = 2,
            extra_distance_between_tick_labels = 15,
            index_start=0,
            index_end=10,
            lw=2,
            ms=20,
            add_legend=False):
    # define index which points should be used
    index = np.arange(index_start, index_end)
    
    # plot true bed_h and surface_h
    ax.plot(total_distance[index],
            true_bed_h[index],
            '.-',
            lw=lw,
            ms=ms,
            c=color_1,
            label=r'true $b$')
    ax.plot(total_distance[index],
            true_sfc_h[index],
            '.--',
            lw=lw,
            ms=ms,
            c=color_1,
            label=r'$s^{end}_o$')
    
    # plot second bed_h and surface_h
    ax.plot(total_distance[index],
            np.append(second_bed_h, true_bed_h[~ice_mask])[index],
            '.-',
            lw=lw,
            ms=ms,
            c=color_2,
            zorder=5,
            label=r'guessed $b$')
    ax.plot(total_distance[index],
            second_sfc_h[index],
            '.--',
            lw=lw,
            ms=ms,
            c=color_2,
            label=r'$s^{end}_m$')
    
    # add glacier polygon
    x_use=total_distance[index]
    x_polygon = np.concatenate((x_use, x_use[::-1]))
    y_polygon = np.concatenate((np.append(second_bed_h, true_bed_h[~ice_mask])[index], 
                                second_sfc_h[index][::-1]))
    coord_polygon = np.concatenate((np.expand_dims(x_polygon, axis=1),np.expand_dims(y_polygon, axis=1)), axis=1)
    ax.add_patch(Polygon(coord_polygon,
                         fc=glacier_color,  
                         ec=None,#outline_color,
                         closed=False,
                         lw = 0.8,
                         zorder=1,
                         label=''))

    if tick_labels is not None:
        # add labels for grid points
        for i, (tick_label, point_index, position)in enumerate(zip(tick_labels,
                                                                   tick_labels_indexes,
                                                                   tick_labels_position)):
            x_use = total_distance[point_index]
            if position == 'above':
                y_use_upper = true_bed_h[point_index] + len_tick_label_line + i * extra_distance_between_tick_labels
                y_use_lower = np.min([true_bed_h[point_index], second_bed_h[point_index]])
                y_use_label = (true_bed_h[point_index] + len_tick_label_line + tick_label_text_y_distance +
                               i * extra_distance_between_tick_labels)
            elif position == 'below':
                y_use_upper = np.max([true_sfc_h[point_index], second_sfc_h[point_index]])
                y_use_lower = true_bed_h[point_index] - len_tick_label_line - i * extra_distance_between_tick_labels
                y_use_label = (true_bed_h[point_index] - len_tick_label_line - tick_label_text_y_distance -
                               i * extra_distance_between_tick_labels)
            # plot line
            ax.plot([x_use, x_use],
                    [y_use_lower, y_use_upper],
                   '-',
                   c=axis_color,
                   lw=lw-1,
                   zorder=1)

            # add tick label text
            ax.text(x_use,
                    y_use_label,
                    tick_label,
                    fontsize=fontsize,
                    c=axis_color,
                    verticalalignment='center',
                    horizontalalignment='center')
    
    # add text with description
    ax.text(0.95, 0.95,
            title,
            fontsize=fontsize,
            verticalalignment='top',
            horizontalalignment='right',
            transform = ax.transAxes)
    # add visual axis
    x_origin = 0.08 #total_distance[index[0]]
    z_origin = 0.08 #true_bed_h[index[-1]]
    x_len = 0.2
    z_len = 0.2
    x_text_setoff = 0.04
    z_text_setoff = 0.04

    # add z axis
    plt.annotate(text='',
                 xy=(x_origin, z_origin), 
                 xytext=(x_origin, z_origin + z_len),
                 xycoords=ax.transAxes,
                 textcoords=ax.transAxes,
                 arrowprops=dict(arrowstyle='<-',
                                 mutation_scale=25,
                                 color=axis_color,
                                 lw=1),
                 zorder=1
                 )
    plt.text(x_origin ,z_origin + z_len + z_text_setoff,'z',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize + 2,
             c=axis_color,
             transform = ax.transAxes)

    # add x axis
    plt.annotate(text='',
                 xy=(x_origin, z_origin), 
                 xytext=(x_origin + x_len, z_origin),
                 xycoords=ax.transAxes,
                 textcoords=ax.transAxes,
                 arrowprops=dict(arrowstyle='<-',
                                 mutation_scale=25,
                                 color=axis_color,
                                 lw=1),
                 zorder=1
                 )
    plt.text(x_origin + x_len + x_text_setoff, z_origin, 'x',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize + 2,
             c=axis_color,
             transform = ax.transAxes)
    # set limits of x and y axis
    #ax.set_xlim([5.5,6.6])
    #ax.set_ylim([2340, 2720])
    
    if add_legend:
        ax.legend(fontsize=fontsize)
    ax.set_xticks([])
    ax.set_yticks([])

## profil legend

In [ ]:
def add_legend(ax,
               title,
               fontsize,
               lw,
               ms,
               labels):
    # plot true bed_h and surface_h
    ax.plot([],
            [],
            '.-',
            lw=lw,
            ms=ms,
            c=color_1,
            label=labels[0])
    ax.plot([],
            [],
            '.--',
            lw=lw,
            ms=ms,
            c=color_1,
            label=labels[1])
    
    # plot second bed_h and surface_h
    ax.plot([],
            [],
            '.-',
            lw=lw,
            ms=ms,
            c=color_2,
            zorder=5,
            label=labels[2])
    ax.plot([],
            [],
            '.--',
            lw=lw,
            ms=ms,
            c=color_2,
            label=labels[3])
    
    ax.legend(loc='center', fontsize=fontsize, title=title)
    ax.axis('off')

# Define Colors

In [ ]:
colors = sns.color_palette("colorblind")
colors

In [ ]:
axis_color = list(colors[7]) + [1.]
color_1 = list(colors[3]) + [1.]
color_2 = list(colors[0]) + [1.]
color_3 = list(colors[4]) + [1.]
color_4 = list(colors[2]) + [1.]
glacier_color = glacier_color = list(colors[9]) + [.5]

# Import Data

In [ ]:
input_folder = 'plot_data/'
filename_50yrs = 'rec_line_wide_ret_spinup_bed_h_50reg10.nc'
filename_100yrs = 'rec_line_wide_ret_spinup_bed_h_100reg10.nc'
filename_150yrs = 'rec_line_wide_ret_spinup_bed_h_150reg10.nc'
filename_200yrs = 'rec_line_wide_ret_spinup_bed_h_200reg10.nc'
filenames = [filename_50yrs,
             filename_100yrs,
             filename_150yrs,
             filename_200yrs]

datasets = []

for filename in filenames:
    with xr.open_dataset(input_folder + filename) as ds:
        datasets.append(ds)

# Create figure comparing all spinup times

In [ ]:
#dataset,
nr_of_iterations = 6
facecolor = 'white'
labels_pad = -240
cmap = 'Spectral'
fontsize = 25
lw=2
ms=20
annotation_x_position_spatial = 0
annotation_x_position_performance = 0
#index_start_first_profil_row = 0
#index_end_first_profil_row = 6
#index_start_second_profil_row = 65
#index_end_second_profil_row = 71
save_file = True
filename = 'compare_different_spinup_times_rec.pdf'

#plt.rcParams['font.family'] = 'monospace'
mpl.rcParams.update({'font.size': fontsize})

fig = plt.figure(figsize=(1,1), facecolor='white')

# define grid
# define fixed size of spatial subplot
spatial_height = 5
spatial_separation_x = 1.5
spatial_width = 3.3
# define fixed size for performance plot
performance_height = 3
separation_y_performance_spatial = 1

# fixed size in inch
# along x axis                                                              x-index for locator
horiz = [Size.Fixed(spatial_width),                                         # 0 first spatial column
         Size.Fixed(spatial_separation_x),
         Size.Fixed(spatial_width),                                         # 2 second spatial column
         Size.Fixed(spatial_separation_x),
         Size.Fixed(spatial_width),                                         # 4 third spatial column
         Size.Fixed(spatial_separation_x),
         Size.Fixed(spatial_width),                                         # 6 fourth spatial column
         ]
                                                                          # y-index for locator
vert = [Size.Fixed(performance_height),                                   # 0 performance row
        Size.Fixed(separation_y_performance_spatial),
        Size.Fixed(spatial_height),                                       # 2 spatial row
       ]

# define indices for subplots for easier changes later
# spatial heatmap
spatial_nx = [0, 2, 4, 6]
spatial_nx1 = [1, 3, 5, 7]
spatial_ny = 2
spatial_annotation = [r'(a) $t^{spinup}$ = 50 a', r'(b) $t^{spinup}$ = 100 a',
                      r'(c) $t^{spinup}$ = 150 a', r'(d) $t^{spinup}$ = 200 a']
# performance
performance_nx = 0
performance_nx1 = 7
performance_ny = 0

# Position of the grid in the figure
rect = (0., 0., 1., 1.)  

# divide the axes rectangle into grid whose size is specified by horiz * vert
divider = Divider(fig, rect, horiz, vert, aspect=False)

for i, one_dataset in enumerate(datasets):
    # add spatial plot
    plot_datasets = convert_iterations_to_dataset(one_dataset)

    with plt.rc_context({'font.family': 'monospace'}):
        ax = setup_axes(fig, 111)
        im = heatmap(plot_datasets[:nr_of_iterations],
                     opti_var='bed_h',
                     annotation=spatial_annotation[i],
                     annotation_x_position=annotation_x_position_spatial,
                     fig=fig,
                     ax=ax,
                     cmap=cmap,
                     grid_color=facecolor,
                     presentation=True,
                     labels_pad=labels_pad,
                     xlim=None,
                     nr_of_iterations=None)
        ax.set_axes_locator(divider.new_locator(nx=spatial_nx[i],
                                                nx1=spatial_nx1[i],
                                                ny=spatial_ny))

ax = fig.subplots()
performance_plot(ax,
                 datasets,
                 nr_of_iterations=nr_of_iterations,
                 annotation=r'(e) evolution of $ELA^{spinup}$ for different spinup times',
                 annotation_x_position=annotation_x_position_performance,
                 lw=lw,
                 fontsize=fontsize,
                 ms=ms)
ax.set_axes_locator(divider.new_locator(nx=performance_nx,
                                        nx1=performance_nx1,
                                        ny=performance_ny))

if save_file:
    fig.savefig(filename, format='pdf', bbox_inches='tight', dpi=300);

# Create profil figure for 50 years spinup

In [ ]:
datasets[0]

## calculate first guess spinup sfc_h

In [ ]:
from oggm.core.massbalance import LinearMassBalance as oggm_MassBalance
from oggm.core.flowline import FluxBasedModel as oggm_FluxModel
from oggm.core.flowline import RectangularBedFlowline

from oggm import cfg
cfg.initialize()

In [ ]:
def calculate_first_guess_spinup_sfc_h(dataset, yrs_to_run):
    map_dx = 100.
    
    spinup_sfc_h = dataset.spinup_sfc_h.data
    bed_h = np.append(dataset.first_guessed_bed_h.data,
                      dataset.total_true_bed_h.data[~dataset.ice_mask.data])
    widths = dataset.true_widths.data / map_dx
    spinup_ELA = dataset.first_guess_spinup_ELA.data
    mb_grad = dataset.attrs['mb_grad'][0]
    
    oggm_fl = RectangularBedFlowline(surface_h=spinup_sfc_h,
                                     bed_h=bed_h,
                                     widths=widths,
                                     map_dx=map_dx)

    oggm_mb_model = oggm_MassBalance(spinup_ELA,
                                     mb_grad)

    model = oggm_FluxModel(oggm_fl,
                           mb_model=oggm_mb_model,
                           y0=0.)
    
    model.run_until(yrs_to_run)
    
    return model.fls[-1].surface_h

## Read data

In [ ]:
with datasets[0] as ds:
    true_bed_h  = ds.total_true_bed_h.data
    true_sfc_h = ds.true_surface_h.data
    true_spinup_sfc_h = ds.true_spinup_sfc_h
    guessed_bed_h = ds.guessed_bed_h.data
    guessed_sfc_h = ds.surface_h.data
    guessed_spinup_sfc_h = ds.spinup_sfc_h_guessed.data
    total_distance = ds.coords['total_distance'].data
    ice_mask = ds.ice_mask.data
    ice_distance = total_distance[ice_mask]
    first_guess_bed_h = ds.first_guessed_bed_h.data
    first_guess_sfc_h = ds.first_guess_surface_h.data
    first_guess_spinup_sfc_h = calculate_first_guess_spinup_sfc_h(ds, 50)

## function to create figure

In [ ]:
def create_profil_plot(
    dataset,
    lw=2,
    ms=0,
    fontsize = 25,
    save_fig = False,
    filename = None,
    start_index = 0,
    end_index = 100,
    spinup_yrs=None,
    tick_labels=None,
    tick_labels_indexes=None,
    tick_labels_position='above',
    len_tick_label_line = 110,
    tick_label_text_y_distance = 2,
    extra_distance_between_tick_labels = 15
    ):
    with dataset as ds:
        true_bed_h  = ds.total_true_bed_h.data
        true_sfc_h = ds.true_surface_h.data
        true_spinup_sfc_h = ds.true_spinup_sfc_h
        guessed_bed_h = ds.guessed_bed_h.data
        guessed_sfc_h = ds.surface_h.data
        guessed_spinup_sfc_h = ds.spinup_sfc_h_guessed.data
        total_distance = ds.coords['total_distance'].data
        ice_mask = ds.ice_mask.data
        ice_distance = total_distance[ice_mask]
        first_guess_bed_h = ds.first_guessed_bed_h.data
        first_guess_sfc_h = ds.first_guess_surface_h.data
        first_guess_spinup_sfc_h = calculate_first_guess_spinup_sfc_h(ds, spinup_yrs)
    fig = plt.figure(figsize=(1,1), facecolor='white')

    # define grid
    # define fixed size of subplot
    subplot_width = 5
    subplot_height = 5
    subplot_separation_x = .1
    subplot_separation_y = .1

    # define height of legend
    legend_width = 3

    #define separation legend subplots
    separation_x_legend_subplots = .1

    # fixed size in inch
    # along x axis                                                              x-index for locator
    horiz = [Size.Fixed(legend_width),                                        # 0 legend
             Size.Fixed(separation_x_legend_subplots),
             Size.Fixed(subplot_width),                                       # 2 1st column subplot
             Size.Fixed(subplot_separation_x),   
             Size.Fixed(subplot_width),                                       # 4 2nd column subplot
             Size.Fixed(subplot_separation_x),
             Size.Fixed(subplot_width),                                       # 6 3rd column subplot
             Size.Fixed(subplot_separation_x),
             Size.Fixed(subplot_width)                                        # 8 4th column subplot
            ]
                                                                                  # y-index for locator
    vert = [Size.Fixed(subplot_height),                                       # 0 2nd row subplot
            Size.Fixed(subplot_separation_y),
            Size.Fixed(subplot_height)                                        # 2 1st row subplot
           ]

    rect = (0., 0., 1., 1.)  # Position of the grid in the figure

    # divide the axes rectangle into grid whose size is specified by horiz * vert
    divider = Divider(fig, rect, horiz, vert, aspect=False)

    # first guess
    ax = fig.subplots()
    profil_subplot(
                ax,
                true_bed_h,
                true_spinup_sfc_h,
                first_guess_bed_h,
                first_guess_spinup_sfc_h,
                '(a) first guess',
                fontsize=fontsize,
                tick_labels=tick_labels,
                tick_labels_indexes=tick_labels_indexes,
                tick_labels_position=tick_labels_position,
                len_tick_label_line = len_tick_label_line,
                tick_label_text_y_distance = tick_label_text_y_distance,
                extra_distance_between_tick_labels = extra_distance_between_tick_labels,
                index_start=start_index,
                index_end=end_index,
                lw=lw,
                ms=ms,
                add_legend=False)
    ax.set_axes_locator(divider.new_locator(nx=2, ny=2))

    ax = fig.subplots()
    profil_subplot(
                ax,
                true_bed_h,
                true_sfc_h,
                first_guess_bed_h,
                first_guess_sfc_h,
                '(e) first guess',
                fontsize=fontsize,
                tick_labels=tick_labels,
                tick_labels_indexes=tick_labels_indexes,
                tick_labels_position=tick_labels_position,
                len_tick_label_line = len_tick_label_line,
                tick_label_text_y_distance = tick_label_text_y_distance,
                extra_distance_between_tick_labels = extra_distance_between_tick_labels,
                index_start=start_index,
                index_end=end_index,
                lw=lw,
                ms=ms,
                add_legend=False)
    ax.set_axes_locator(divider.new_locator(nx=2, ny=0))

    # add iteration 1, 2, 3
    for i, prefix in zip(np.arange(1,4), [['(b) ', '(e) '],
                                          ['(c) ', '(f) '],
                                          ['(d) ', '(g) '],]):
        ax = fig.subplots()
        profil_subplot(
                ax,
                true_bed_h,
                true_spinup_sfc_h,
                guessed_bed_h[i-1],
                guessed_spinup_sfc_h[i-1],
                title=prefix[0] + str(i) + '. Iteration',
                fontsize=fontsize,
                tick_labels=tick_labels,
                tick_labels_indexes=tick_labels_indexes,
                tick_labels_position=tick_labels_position,
                len_tick_label_line = len_tick_label_line,
                tick_label_text_y_distance = tick_label_text_y_distance,
                extra_distance_between_tick_labels = extra_distance_between_tick_labels,
                index_start=start_index,
                index_end=end_index,
                lw=lw,
                ms=ms,
                add_legend=False)
        ax.set_axes_locator(divider.new_locator(nx=(i + 1)*2, ny=2))

        ax = fig.subplots()
        profil_subplot(
                ax,
                true_bed_h,
                true_sfc_h,
                guessed_bed_h[i-1],
                guessed_sfc_h[i-1],
                title=prefix[1] + str(i) + '. Iteration',
                fontsize=fontsize,
                tick_labels=tick_labels,
                tick_labels_indexes=tick_labels_indexes,
                tick_labels_position=tick_labels_position,
                len_tick_label_line = len_tick_label_line,
                tick_label_text_y_distance = tick_label_text_y_distance,
                extra_distance_between_tick_labels = extra_distance_between_tick_labels,
                index_start=start_index,
                index_end=end_index,
                lw=lw,
                ms=ms,
                add_legend=False)
        ax.set_axes_locator(divider.new_locator(nx=(i + 1)*2, ny=0))

    # add legend at t spinup
    labels= [r'$b_{t}$', r'$s^{spinup}_{o}$', r'$b$', r'$s^{spinup}_{m}$']
    ax = fig.subplots()
    add_legend(ax=ax,
               title='For (a) - (d)',
               fontsize=fontsize,
               lw=lw,
               ms=ms,
               labels=labels)
    ax.set_axes_locator(divider.new_locator(nx=0, ny=2))
    
    # add legend at t end
    labels= [r'$b_{t}$', r'$s^{e}_{o}$', r'$b$', r'$s^{e}_{m}$']
    ax = fig.subplots()
    add_legend(ax=ax,
               title='For (e) - (h)',
               fontsize=fontsize,
               lw=lw,
               ms=ms,
               labels=labels)
    ax.set_axes_locator(divider.new_locator(nx=0, ny=0))

    if save_fig:
        fig.savefig(filename,format='pdf',bbox_inches='tight',dpi=300);

## Test for different spinup years

### 50 years

In [ ]:
dataset = datasets[0]
spinup_yrs = 50
filename = 'profil_50_year_spinup_time.pdf'
lw = 2
ms = 0
fontsize = 30
start_index = 35
end_index = 95
save_fig = True
tick_labels=['trm']
tick_labels_indexes=[84]
tick_labels_position=['above']
len_tick_label_line = 400
tick_label_text_y_distance = 100
extra_distance_between_tick_labels = 0


create_profil_plot(
    dataset=dataset,
    lw=lw,
    ms=ms,
    fontsize = fontsize,
    save_fig = save_fig,
    filename = filename,
    start_index = start_index,
    end_index = end_index,
    spinup_yrs=spinup_yrs,
    tick_labels=tick_labels,
    tick_labels_indexes=tick_labels_indexes,
    tick_labels_position=tick_labels_position,
    len_tick_label_line = len_tick_label_line,
    tick_label_text_y_distance = tick_label_text_y_distance,
    extra_distance_between_tick_labels = extra_distance_between_tick_labels,
    )

### 100 years

In [ ]:
dataset = datasets[1]
spinup_yrs = 100
filename = 'profil_100_year_spinup_time.pdf'
lw = 2
ms = 0
fontsize = 30
start_index = 35
end_index = 95
save_fig = False
tick_labels=['36', '85/trm']
tick_labels_indexes=[35, 84]
tick_labels_position=['below', 'above']
len_tick_label_line = 400
tick_label_text_y_distance = 100
extra_distance_between_tick_labels = 0


create_profil_plot(
    dataset=dataset,
    lw=lw,
    ms=ms,
    fontsize = fontsize,
    save_fig = save_fig,
    filename = filename,
    start_index = start_index,
    end_index = end_index,
    spinup_yrs=spinup_yrs,
    tick_labels=tick_labels,
    tick_labels_indexes=tick_labels_indexes,
    tick_labels_position=tick_labels_position,
    len_tick_label_line = len_tick_label_line,
    tick_label_text_y_distance = tick_label_text_y_distance,
    extra_distance_between_tick_labels = extra_distance_between_tick_labels,
    )

### 150 years

In [ ]:
dataset = datasets[2]
spinup_yrs = 150
filename = 'profil_150_year_spinup_time.pdf'
lw = 2
ms = 0
fontsize = 25
start_index = 0
end_index = 100
save_fig = False


create_profil_plot(
    dataset=dataset,
    lw=lw,
    ms=ms,
    fontsize = fontsize,
    save_fig = save_fig,
    filename = filename,
    start_index = start_index,
    end_index = end_index,
    spinup_yrs=spinup_yrs
    )

### 200 years

In [ ]:
dataset = datasets[3]
spinup_yrs = 200
filename = 'profil_200_year_spinup_time.pdf'
lw = 2
ms = 0
fontsize = 25
start_index = 0
end_index = 100
save_fig = False


create_profil_plot(
    dataset=dataset,
    lw=lw,
    ms=ms,
    fontsize = fontsize,
    save_fig = save_fig,
    filename = filename,
    start_index = start_index,
    end_index = end_index,
    spinup_yrs=spinup_yrs
    )